# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.232938973036                   -0.50    7.0
  2   -7.249608802789       -1.78       -1.38    1.0
  3   -7.251195718313       -2.80       -1.88    4.0
  4   -7.251177396220   +   -4.74       -2.03    2.0
  5   -7.251288410203       -3.95       -2.32    2.0
  6   -7.251335612995       -4.33       -2.84    2.0
  7   -7.251338697727       -5.51       -3.71    1.0
  8   -7.251338770710       -7.14       -3.83    4.0
  9   -7.251338795066       -7.61       -4.29    2.0
 10   -7.251338798289       -8.49       -4.76    2.0
 11   -7.251338798672       -9.42       -5.34    3.0
 12   -7.251338798696      -10.62       -5.58    2.0
 13   -7.251338798704      -11.11       -6.18    2.0
 14   -7.251338798705      -12.14       -6.72    3.0
 15   -7.251338798705      -13.52       -6.98    2.0
 16   -7.251338798705      -14.45       -7.39    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06616836826413232
[ Info: Arnoldi iteration step 2: normres = 0.6669242337922955
[ Info: Arnoldi iteration step 3: normres = 0.6603517036748381
[ Info: Arnoldi iteration step 4: normres = 0.24555860761196047
[ Info: Arnoldi iteration step 5: normres = 0.3976138552028982
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.16e-02, 6.13e-02, 2.91e-01, 2.64e-01, 1.09e-02)
[ Info: Arnoldi iteration step 6: normres = 0.42312028359556975
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.27e-03, 2.03e-01, 3.37e-01, 1.04e-01, 1.12e-01)
[ Info: Arnoldi iteration step 7: normres = 0.05665115049629118
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.29e-04, 1.72e-02, 9.11e-03, 2.81e-02, 3.98e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12109143818343916
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.75e